In [ ]:

from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_from_disk
import numpy as np
from processingDatasetDotless import *
import torch
from transformers import pipeline
from tashaphyne.stemming import ArabicLightStemmer

In [ ]:
dataset = load_from_disk("/Users/ammar/Developer/git-repos/dotless/Models/Model v3/AR-multi-dotted-Small-arrow")


In [ ]:
dataset

In [ ]:
dataset['train'][20]

In [ ]:
indicies = []
for i in range(10000):
    length = len(dataset['train'][i]["Options"])
    if length > 10 and length <= 20:
        indicies.append(i)

print(indicies)
print(len(indicies))
dataset['train'][indicies[3]]


In [ ]:
with open("/Users/ammar/Developer/git-repos/dotless/Models/Model v4/roots.txt", 'r') as file:
    wordlist = set(file.read().split())


In [ ]:
"#" in list(wordlist)

In [ ]:
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [ ]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

In [ ]:
pipe = pipeline("fill-mask", model="UBC-NLP/ARBERTv2")

In [ ]:
pipe("اللغة [MASK] هي لغة العرب")


In [ ]:
ArListem = ArabicLightStemmer()


In [ ]:
word = u'مؤمنون'

In [ ]:
tokenized_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
tokenized_word

In [ ]:
word_tokenID = tokenizer.convert_ids_to_tokens(tokenized_word)

In [ ]:
stem = ArListem.light_stem(word)
root = ArListem.get_root()
root

In [ ]:
root in wordlist

In [ ]:
def get_candidate_word_probabilities(input_text, candidate_words):
    tokenized_text = tokenizer.tokenize(input_text)
    masked_word_index = tokenized_text.index('[MASK]')
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

    with torch.no_grad():
        output = model(input_ids)

    predictions = output.logits[0, masked_word_index].softmax(dim=0)

    # Tokenize and convert all candidate words to ids at once
    
    tokenized_candidate_words = [tokenizer.tokenize(word)for word in candidate_words]
    #print(tokenized_candidate_words)
    
    tokenized_candidate_words = [word for word in tokenized_candidate_words if len(word) == 1]
    #print(tokenized_candidate_words)
    
    flattened_tokenized_candidate_words = [word for sublist in tokenized_candidate_words for word in sublist]
    #print(flattened_tokenized_candidate_words)
    
    candidate_word_ids = [tokenizer.convert_tokens_to_ids(TCW) for TCW in flattened_tokenized_candidate_words]
    #print(candidate_word_ids)
    
    # Use a list comprehension to populate the dictionary
    candidate_probabilities = {word: predictions[word_id].item() for word, word_id in zip(flattened_tokenized_candidate_words, candidate_word_ids)}
    #print(candidate_probabilities)

    return candidate_probabilities


def generate_probabilties_end(example):
    input_text = example["Masked"]
    candidates = example["Options"]

            
    word_probabilities = get_candidate_word_probabilities(input_text, candidates)

    sorted_words = sorted(word_probabilities, key=word_probabilities.get, reverse=True)
    if len(sorted_words ) > 0:
        most_probable_word = sorted_words[0]
    else:
        most_probable_word = None
        #print(example["Target"])
        
    return word_probabilities, sorted_words, most_probable_word


In [ ]:
num_eg = 0
#example = dataset['train'][indicies[num_eg]]
example = mask_random_word("قرأ")
word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example)
print("Length of words:", len(sorted_words))
for word in sorted_words:
    probability = word_probabilities[word]
    print(f"Word: '{word}', Probability: {probability:.10f}")
    
print()

print("Most probable word:", most_probable_word)
print("Target word:", example["Target"])
print("------------------------------------------")

found = False
for i in range(len(sorted_words)):
    if sorted_words[i] == example["Target"]:
        print("Sucess at probability level:", i)
        found = True
        break
if not found: print("Not found.")
sucess_level = i

example

In [ ]:

count = np.zeros(1000000, dtype =int)
#longWords = []
#veryLongWords = []
#veryVeryLongWords = []
start = 0
end = 101
iterationCount = end - start - 1
for i in range(start, end):
    num_eg = i
    #example1 = mask_random_word("التحدي الصعب يكمن في فهم التفاصيل الدقيقة لهذه الفكرة المعقدة")
    #example1 = dataset['train'][indicies[num_eg]]
    example1 = dataset['train'][num_eg]

    word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example1)
    for word in sorted_words:
        probability = word_probabilities[word]
        
    #if len(sorted_words) >= 500 and len(sorted_words) < 1000: longWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000 and len(sorted_words) < 100000: veryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 100000 and len(sorted_words) <1000000: veryVeryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000000: veryVeryLongWords.append(example1["Target"])



    for j in range(len(sorted_words)):
        if sorted_words[j] == example1["Target"]:
            count[j] += 1
            break
        

for k in range(len(count)):
    numberCount = count[k]
    if numberCount != 0:
        print(numberCount, "placments at level", k, "  \t", (numberCount/iterationCount) * 100, "%")
        
print()
print("Number of examples tested:", iterationCount)
print("Number of examples that yielded a result:", np.sum(count))


In [ ]:
def mask_word(text, index=None):

    words = text.split()
    if index == None: index = random.randint(0, len(words) - 1)
    
    for i, word in enumerate(words):
        if i == index:
            word_to_replace = words[i]
            variations = allVariation(parse_dotless_text(word_to_replace))
            words[i] = "[MASK]"
            modified_string = ' '.join(words)
            masked = (modified_string)
            options = (variations)
            targets = (word_to_replace)
    
    return {"Masked": masked, "Options": options, "Target": targets}

In [ ]:

mask_word("قنام", 0)

    

In [ ]:
parse_dotless_text("قرأ أحمد كتابه ثم نام")

In [ ]:


import tkinter as tk

def function123(input_text, cursor_position):
    
    # Get the text up to the cursor position
    text_up_to_cursor = input_text[:cursor_position] 

    # Split the text into words
    words = text_up_to_cursor.split()

    # The word number that the user is currently editing is the number of words
    word_number = len(words) - 1
    current_word = words[word_number]
    
    print("current_word:", current_word)
    print("Cursor position:", cursor_position)
    print("Word Number:", word_number)
    print("input_text:", input_text)
    
    Fexample =  mask_word(input_text, word_number)
    Fword_probabilities, Fsorted_words, Fmost_probable_word = generate_probabilties_end(Fexample)
    Msentence = Fexample["Masked"]
    
    if Fmost_probable_word == None:
        fill = words[word_number]
    else:
        fill = Fmost_probable_word
        
    sentence = Msentence.replace("[MASK]", fill)
    

    print("example:",Fexample)
    print("most_probable_word:", Fmost_probable_word)
    print("fill:", fill)
    print("sentence:", sentence)
    print("--------------------------------------------------")
    return sentence


def update_input(event):
    
    # Ignore non-character key presses
    if event.char == "" or event.keysym == "space":
        return

    # Get the current text input from the Entry widget
    input_text = entry.get()

    # Call the function123() with the current input and get the processed string
    cursor_position = entry.index(tk.INSERT)

    processed_text = function123(input_text, cursor_position)

    # Replace the text in the Entry widget with the processed string
    entry.delete(0, tk.END)
    entry.insert(0, processed_text)

# Create the main window
main = tk.Tk()
main.geometry("800x600")
# Create the Entry widget
entry = tk.Entry(main)

# Bind the <Key> event to the update_input function

entry.bind("<KeyRelease>", update_input)

# Pack the Entry widget to display it in the window
entry.pack()

# Start the main event loop
main.mainloop()